https://github.com/OpenDataDE/State-zip-code-GeoJSON

# Imports

In [1]:
import pandas as pd
import folium
import json
import os

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

# Configs

In [3]:
SELECTED_NAME = "RDC"

In [4]:
geojson_dir = r"D:\GIT\zipmaps"
geojson = {
    "DC":"dc_district_of_columbia_zip_codes_geo.min.json",
    "VA":"va_virginia_zip_codes_geo.min.json",
    "MD":"md_maryland_zip_codes_geo.min.json",
}

In [5]:
MAP_SAVE_NAME = '{}_ZipMap.html'

In [6]:
deliv_url = r"https://delivery.deliv.co/zones"

In [7]:
chromedirver_dir = r"D:\GIT\zipmaps"

In [8]:
chromedriver_path = os.path.join(chromedirver_dir, "chromedriver.exe")

In [9]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless");

driver = webdriver.Chrome(chromedriver_path, chrome_options=chrome_options)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


# Functions

In [10]:
def retrieveZipData(zipcode, driver=driver):
    zipcode = str(zipcode)
    
    driver.get(deliv_url)
    inputElement = driver.find_element_by_id("origin_zip")
    inputElement.clear()
    inputElement.send_keys(zipcode)
    get_div = driver.find_element_by_tag_name('button')
    get_div.click()

    soup = bs(driver.page_source, 'lxml')

    zip_data = {
        "MAPDATA":{},
        "SELECTED":zipcode
    }
    for zone in soup.find_all("div", class_="zone-pricing-results quote-border row"):
        c_name = zone.find("div", class_="zone-name columns small-4").text.strip()
        c_price = float(zone.find("span", class_="zone-price").text.strip().replace("$", ""))
        c_zips = zone.find("p", class_="zone-list").text.strip()
        zip_data["MAPDATA"][c_name]={"price":c_price, "zipcodes":[i.strip() for i in c_zips.split(",")]}
    
    return zip_data

In [11]:
def getZipData(zipcode):
    zipcode = str(zipcode)
    if os.path.isfile(os.path.join(geojson_dir, "delivery.json")):
        with open(os.path.join(geojson_dir, "delivery.json"), "r") as delivery_file:
            delivery_dict = json.load(delivery_file)
    else:
        delivery_dict = {}

    if zipcode not in delivery_dict:
        delivery_dict[zipcode] = retrieveZipData(zipcode)     
        with open(os.path.join(geojson_dir, "delivery.json"), "w+") as delivery_file:
            delivery_file.write(json.dumps(delivery_dict, indent=4, sort_keys=True))
    
    return delivery_dict[zipcode]

In [12]:
def makeDF(zipcode, max_price=100):
    zipcode = str(zipcode)
    dfdata = [[k, v["price"], c_zip] for (k,v) in getZipData(zipcode)["MAPDATA"].items() for c_zip in v["zipcodes"]]
    dfdata = [i for i in dfdata if i[1]<=max_price]
    #     base_price = getZipData(zipcode)["MAPDATA"]['Zone 1']['price']
    base_price = 0
    selectedDF = pd.DataFrame([[SELECTED_NAME, base_price, zipcode]], columns=["ZONE", "PRICE", "ZIPCODE"])
    zipDF = pd.DataFrame(dfdata, columns=["ZONE", "PRICE", "ZIPCODE"])
    zipDF = zipDF[zipDF["ZIPCODE"] != zipcode]
    zipDF = selectedDF.append(zipDF, ignore_index = True)
    return zipDF

In [13]:
def getGeoJSON(zipcodes_of_interest):
    geojson_data = {k:json.load(open(os.path.join(geojson_dir, v), 'r')) for (k,v) in geojson.items()}

    combined_geojson = {'type': 'FeatureCollection', 'features': []}
    for state, data in geojson_data.items():
        combined_geojson['features'].extend(data['features'])

    combined_geojson['features'] = [i for i in combined_geojson['features'] if i["properties"]["ZCTA5CE10"] in zipcodes_of_interest]
    
    return combined_geojson

In [18]:
def tooltipText(destination_zipcode, zipDF):
    price = list(zipDF[zipDF["ZIPCODE"]==destination_zipcode].PRICE)[0]
    if price == 0:
        price = min(list(zipDF[zipDF["PRICE"]!=0].PRICE))
    return "zipcode: {}\nPrice: ${}".format(
        destination_zipcode,
        price
    )

In [19]:
def makeMap(zipcode, max_price=100):
    
    zipDF = makeDF(zipcode, max_price)
    combined_geojson = getGeoJSON(list(zipDF.ZIPCODE))
    
    m = folium.Map(location=[38.9,-77.2], zoom_start=11)

    folium.Choropleth(
        geo_data=combined_geojson,
        fill_opacity=.7,
        line_opacity=.2,
        data=zipDF,
        columns=["ZIPCODE", "PRICE"],
        key_on='feature.properties.ZCTA5CE10',
        fill_color='YlOrRd',
        highlight=True
    ).add_to(m)
    

    for feature in combined_geojson['features']:
        c_point = (float(feature["properties"]["INTPTLAT10"]), float(feature["properties"]["INTPTLON10"]))
        folium.Marker(c_point, popup=tooltipText(feature["properties"]["ZCTA5CE10"], zipDF)).add_to(m)

    m.save(os.path.join(geojson_dir, MAP_SAVE_NAME.format(zipcode)))
    return m

# User Inputs / Running

In [20]:
makeMap(22209, 20)